In [1]:
import numpy as np
import pandas as pd
import re
import torch
import torch.nn as nn
import torchtext
import torecsys as trs
from typing import Dict, Tuple, List

c:\program files\python37\lib\site-packages\tqdm\autonotebook\__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
c:\program files\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\program files\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\program files\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning:

In [42]:
import logging

In [43]:
x = logging.Logger()

TypeError: __init__() missing 1 required positional argument: 'name'

In [39]:
pathlib.Path("./log_dir").mkdir(parents=True, exist_ok=True)

In [2]:
# get samples data from movielens as a example
# _, movies_df, ratings_df, _ = trs.data.sampledata.load_ml_data(size="latest-small")
# movies_df["year"] = movies_df.title.apply(lambda x: re.findall(r"\((\d+)\)", x))
# movies_df["year"] = movies_df.year.apply(lambda x: int(x[0]) if len(x) > 0 else np.nan)
# movies_df = pd.concat([
#     movies_df, 
#     pd.get_dummies(movies_df.genres.apply(
#         lambda x: x.split("|")).apply(pd.Series).stack()).sum(level=0)
# ], axis=1).drop(["title", "genres"], axis=1)
# merged = pd.merge(ratings_df, movies_df, on="movieId")

# Inputs

In [ ]:
emb_dict = torch.jit.trace(trs.inputs.base.EmbeddingDict(["userId", "itemId"], ["single_index", "single_index"], [100, 50], [16, 16]), {"userId": (torch.Tensor([[1]]).long()), "itemId": (torch.Tensor([[1]]).long())})
emb_dict({"userId": (torch.Tensor([[1]]).long()), "itemId": (torch.Tensor([[1]]).long())})

In [ ]:
fa_emb = torch.jit.trace(trs.inputs.base.FieldAwareMultipleIndexEmbedding(16, [100, 10]), torch.Tensor([[1, 1]]).long())
fa_emb(torch.Tensor([[1, 1]]).long())

In [ ]:
list_emb = torch.jit.trace(trs.inputs.base.ListIndexEmbedding(16, 100, num_heads=4), torch.Tensor([[1, 4, 5]]).long())
list_emb(torch.Tensor([[1, 4, 5]]).long())
# try to fix if use trace
# list_emb.show_attention(torch.Tensor([[1, 4, 5]]).long())

In [ ]:
mlp_emb = torch.jit.trace(trs.inputs.base.MultipleIndexEmbedding(16, [100, 10]), torch.Tensor([[1, 1]]).long())
mlp_emb(torch.Tensor([[1, 1]]).long())

In [ ]:
sq_emb = torch.jit.trace(trs.inputs.base.SequenceIndexEmbedding(16, 100), (torch.Tensor([[3, 1, 0], [4, 2, 5]]).long(), torch.Tensor([2, 3]).long()))
sq_emb(torch.Tensor([[3, 1, 0], [4, 2, 5]]).long(), torch.Tensor([2, 3]).long())

In [ ]:
index_emb = torch.jit.trace(trs.inputs.base.SingleIndexEmbedding(16, 100), torch.Tensor([[1]]).long())
index_emb(torch.Tensor([[1]]).long())

In [ ]:
stack = torch.jit.trace(trs.inputs.base.StackedInputs(["userId", "itemId"], ["single_index", "single_index"], [100, 50], [16, 16]), ({"userId": (torch.Tensor([[1]]).long()), "itemId": (torch.Tensor([[1]]).long())}))
stack({"userId": (torch.Tensor([[1]]).long()), "itemId": (torch.Tensor([[1]]).long())})

In [ ]:
val = torch.jit.trace(trs.inputs.base.ValueInputs(5), torch.randn(size=(1, 5)))
val(torch.randn(size=(1, 5)))

In [ ]:
img_inputs = torch.jit.trace(trs.inputs.base.ImageInputs(16, 3, [8, 8], [1, 1], [1, 1], [1, 1]), torch.randn(4, 3, 64, 64))
img_inputs(torch.randn(4, 3, 64, 64))

In [ ]:
pre_img = torch.jit.trace(trs.inputs.base.PretrainedImageInputs(16, "resnet18"), torch.randn(4, 3, 64, 64))
pre_img(torch.randn(4, 3, 64, 64))

In [24]:
inputs = {
    "sequenceId"     : torch.randint(100, size=(8, 4)),
    "sequenceLength" : torch.Tensor([4] * 8),
    "values"         : torch.randn(size=(8, 8)),
    "userId"         : torch.randint(100, size=(8, 1))
}

In [25]:
cat_schema = [
    (trs.inputs.base.SequenceIndexEmbedding(8, 100), ["sequenceId"], ["sequenceLength"]),
    (trs.inputs.base.ValueInputs(8), ["values"])
]

In [26]:
stacked_schema = [
    (trs.inputs.base.SingleIndexEmbedding(16, 100), ["userId"]), 
    (trs.inputs.base.ConcatInputs(cat_schema), ["sequenceId", "sequenceLength", "values"])
]

In [27]:
stacked = trs.inputs.base.StackedInputs(stacked_schema)

In [29]:
wrapped_schema = {
    "stacked": (trs.inputs.base.StackedInputs(stacked_schema), ["userId", "sequenceId", "sequenceLength", "values"])
}

In [31]:
wrapped = trs.inputs.InputsWrapper(wrapped_schema)

In [32]:
wrapped(inputs)

{'stacked': tensor([[[-0.8068,  1.8913,  0.9308, -0.2586, -0.8677,  0.8212,  0.5583,
           -0.4547,  0.7802, -0.2594,  0.4328, -0.5197, -0.1188,  0.6041,
           -0.1233,  0.6427],
          [ 0.0865,  0.0638, -0.1338, -0.2038,  0.1361,  0.2313,  0.0161,
            0.1685, -0.8561,  0.6156,  1.0580, -0.1640,  2.1856, -0.0967,
           -0.4956,  0.5661]],
 
         [[ 0.0071, -0.2882, -1.2425, -0.1356,  0.6158, -2.0084, -1.1312,
           -0.0764, -0.4758, -0.0837,  0.6379, -1.7833, -0.7717, -1.1022,
           -0.6417, -0.5653],
          [ 0.1331,  0.1828, -0.0099, -0.3414,  0.1036,  0.2049, -0.0202,
            0.1084, -1.7258,  1.1650, -0.5486,  0.2988,  0.3021, -1.3602,
           -0.6615,  0.7942]],
 
         [[ 1.6530, -1.1351, -0.1689,  0.6209,  0.7906,  1.5958,  0.6635,
            1.3527,  0.5280, -1.3512,  1.3713, -0.7543,  1.9354, -0.7788,
           -0.8546,  0.1996],
          [ 0.1855,  0.2000, -0.0851, -0.2682,  0.1109,  0.1906, -0.0362,
            0.2185,

# Example of Field-aware Factorization Machine with trs.Trainer

In [8]:
embed_size = 16
num_fields = 2

userIdSize = 100
itemIdSize = 100

# inititalize embedding fields
feat_inputs_embedding = trs.inputs.base.MultipleIndexEmbedding(
    1, [userIdSize, itemIdSize]
)
field_aware_embedding = trs.inputs.base.FieldAwareMultipleIndexEmbedding(
    embed_size, [itemIdSize, itemIdSize]
)

# define schema of wrapper and initialize InputsWrapper
schema = {
    "feat_inputs"      : (feat_inputs_embedding, ["userId", "movieId"]),
    "field_emb_inputs" : (field_aware_embedding, ["userId", "movieId"])
}
inputs_wrapper = trs.inputs.InputsWrapper(schema)

# initialize field-aware factorizatiob machine model
ffm = trs.models.FieldAwareFactorizationMachineModel(embed_size, num_fields)

In [16]:
isinstance(ffm, trs.models._Model)

True

In [9]:
inputs = dict(
    userId  = torch.randint(100, size=(16, 1)), 
    movieId = torch.randint(100, size=(16, 1))
)

In [10]:
# embedding
embed_inputs    = inputs_wrapper(inputs)

# prediction by model
output = ffm(**embed_inputs)
print(output)

tensor([[ 4.0732e-01],
        [ 2.4323e-01],
        [-7.9130e-01],
        [-4.9572e-01],
        [-8.0709e-01],
        [ 1.9575e-01],
        [-9.8850e-01],
        [ 3.2924e-01],
        [-2.6344e-01],
        [ 2.4496e+00],
        [-1.2063e+00],
        [ 5.6532e-01],
        [-3.0166e-04],
        [-5.1893e-01],
        [-1.2735e-01],
        [ 1.0425e+00]], grad_fn=<AddBackward0>)


# Models

In [ ]:
e = 8
n = 5
b = 16

feat_inputs = torch.randn(size=(b, n, 1))
field_emb = torch.randn(size=(b, n, e))
field_aware_emb = torch.randn(size=(b, n * n, e))

In [ ]:
user_item_emb = torch.randn(size=(b, 2, e))

In [ ]:
afm_model = torch.jit.trace(trs.models.AttentionalFactorizationMachineModel(e, n, 8, 0.0), (feat_inputs, field_emb))
afm_model(feat_inputs, field_emb)

In [ ]:
dc_model = torch.jit.trace(trs.models.DeepAndCrossNetworkModel(e * n, 1, [10], 2, 1, [0.0], nn.ReLU6()), field_emb)
dc_model(field_emb)

In [ ]:
dffm_model = torch.jit.trace(trs.models.DeepFieldAwareFactorizationMachineModel(e, n, 1, [10], 0.0, [0.0], nn.Tanh(), 1), field_aware_emb)
dffm_model(field_aware_emb)

In [ ]:
dfm_model = torch.jit.trace(trs.models.DeepFactorizationMachineModel(e, n, [10], 0.0, [0.0], nn.ReLU(), 1), (feat_inputs, field_emb))
dfm_model(feat_inputs, field_emb)

In [ ]:
fm = torch.jit.trace(trs.models.FactorizationMachineModel(e, n, 0.1), (feat_inputs, field_emb))
fm(feat_inputs, field_emb)

In [ ]:
fmnn = torch.jit.trace(trs.models.FactorizationMachineSupportedNeuralNetwork(e, n, 1, [10], 0.1, [0.1], nn.Tanh()), (feat_inputs, field_emb))
fmnn(feat_inputs, field_emb)

In [ ]:
fatdffm = torch.jit.trace(trs.models.FieldAttentiveDeepFieldAwareFactorizationMachineModel(e, n, 10, [10], 16, 0.0, [0.1], nn.Tanh()), field_aware_emb)
fatdffm(field_aware_emb)

In [ ]:
ffm = torch.jit.trace(trs.models.FieldAwareFactorizationMachineModel(e, n), (feat_inputs, field_aware_emb))
ffm(feat_inputs, field_aware_emb)

In [ ]:
lr = torch.jit.trace(trs.models.LogisticRegressionModel(n), feat_inputs)
lr(feat_inputs)

In [ ]:
ncf = torch.jit.trace(trs.models.NeuralCollaborativeFilteringModel(e, 1, [10], [0.1], nn.ReLU6()), user_item_emb)
ncf(user_item_emb)

# Layers

In [ ]:
afm = trs.layers.AttentionalFactorizationMachineLayer(e, n, attn_size=8, dropout_p=0.0)
print(afm(field_emb)[0].size())
print(afm(field_emb)[1].size())

In [ ]:
blin = trs.layers.BilinearNetworkLayer(e, n, 1, 3)
blin(field_emb).size()

In [ ]:
cen = trs.layers.ComposeExcitationNetworkLayer(n)
cen(field_aware_emb).size()

In [ ]:
cin = trs.layers.CompressInteractionNetworkLayer(e, n, 1, [10, 10, 10])
cin(field_emb).size()

In [ ]:
cross = trs.layers.CrossNetworkLayer(5, e, n)
cross(field_emb).size()

In [ ]:
fm = trs.layers.FactorizationMachineLayer(0.0)
fm(field_emb).size()

In [ ]:
ffm = trs.layers.FieldAwareFactorizationMachineLayer(n, 0.0)
ffm(field_aware_emb).size()

In [ ]:
inner = trs.layers.InnerProductNetworkLayer(n)
inner(field_emb).size()

In [ ]:
mlp = trs.layers.MultilayerPerceptronLayer(1, [10, 10, 10], embed_size=e, num_fields=n, dropout_p=[0.0, 0.0, 0.0])
mlp(field_emb).size()

In [ ]:
outer = trs.layers.OuterProductNetworkLayer(e, n)
outer(field_emb).size()

In [ ]:
wide = trs.layers.WideLayer(e, n, 1)
wide(field_emb).size()